# Ensemble Learning

## Initial Imports

In [69]:
import warnings
warnings.filterwarnings('ignore')

In [70]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [71]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [72]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [73]:
# Create dummy variables for categorical variables
df_encoded = pd.get_dummies(df, columns=['home_ownership', 'verification_status', 'issue_d', 'pymnt_plan', 'initial_list_status', 'application_type', 'hardship_flag', 'debt_settlement_flag'])

# Drop next_pymt_d' column as irrelevant for credit risk
df_encoded.drop('next_pymnt_d', axis=1, inplace=True)

df_encoded.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,0,0,1,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,0,0,1,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,0,0,1,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,0,0,1,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,0,0,1,1,0,1,1,0,1,1


In [75]:
# Create our features
X = df_encoded.drop('loan_status', axis=1)

# Create our target
y = df_encoded['loan_status']
X.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,0,0,1,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,0,0,1,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,0,0,1,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,0,0,1,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,0,0,1,1,0,1,1,0,1,1


In [76]:
# Check the balance of our target values
Counter(y)

Counter({'low_risk': 68470, 'high_risk': 347})

In [77]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [78]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler()

In [79]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler.fit(X_train)

StandardScaler()

In [80]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled[1]

array([-0.7493051 , -1.08086564, -0.69775565, -0.38502671, -0.25660592,
        1.09157521, -0.65473938,  0.56805993, -0.37456588, -0.29557798,
       -0.37805156, -0.74883285, -0.74837353, -0.33715221, -0.33697752,
       -0.19146786, -0.69651339, -0.03351112,  0.        ,  0.        ,
       -0.30150421, -0.12149415,  0.        ,  0.        ,  0.02776144,
       -0.64143964, -0.86782359,  2.26116306, -0.86828659, -0.58599503,
       -0.02569086, -0.03523963,  1.47426402, -0.84533094, -0.24818917,
        0.66617718,  1.1564938 , -0.39317584, -0.89496311, -0.58955494,
       -0.91106467, -0.57982554, -0.67545218, -0.16534073, -0.13956394,
       -0.07969493, -0.00899006,  0.96108716, -1.23003324,  0.15940589,
        1.08092823, -0.82547384,  0.22449614,  2.03331802, -0.33784239,
       -0.73260843, -0.76504443, -0.66170453, -0.96407603,  0.59577387,
       -0.52117436, -0.93787143, -0.76610915,  0.57070665,  0.        ,
        0.        , -0.13469636, -1.17043341, -0.00597839,  0.08

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [81]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(random_state=1)

In [82]:
# Calculated the balanced accuracy score
y_pred = brf.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.7581814571775232

In [83]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   66,    35],
       [ 2345, 14759]])

In [84]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.65      0.86      0.05      0.75      0.55       101
   low_risk       1.00      0.86      0.65      0.93      0.75      0.58     17104

avg / total       0.99      0.86      0.65      0.92      0.75      0.58     17205



In [95]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_

# We can sort the features by their importance
sorted(zip(importances, X.columns), reverse=True)[:20]

[(0.0731885912769716, 'total_rec_prncp'),
 (0.060925386731491366, 'total_pymnt_inv'),
 (0.05995925273012057, 'total_rec_int'),
 (0.055767786194095746, 'last_pymnt_amnt'),
 (0.04924801161828595, 'total_pymnt'),
 (0.029319146728817382, 'int_rate'),
 (0.024776542703796846, 'issue_d_Jan-2019'),
 (0.022352465540959317, 'installment'),
 (0.01930757916648539, 'mths_since_recent_inq'),
 (0.01885280468016428, 'avg_cur_bal'),
 (0.018380838517325936, 'max_bal_bc'),
 (0.01651405759620199, 'out_prncp_inv'),
 (0.01587071998557892, 'total_bal_il'),
 (0.015662708985492955, 'tot_cur_bal'),
 (0.015434973593084727, 'dti'),
 (0.015375069032892052, 'out_prncp'),
 (0.01529552006213333, 'mths_since_rcnt_il'),
 (0.015043314278468799, 'mo_sin_old_rev_tl_op'),
 (0.01492221617266548, 'revol_bal'),
 (0.014375308430773039, 'annual_inc')]

### Easy Ensemble Classifier

In [96]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(random_state=42)
eec.fit(X_train_scaled, y_train) 

EasyEnsembleClassifier(random_state=42)

In [97]:
# Calculated the balanced accuracy score
y_pred = eec.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.8317665255767859

In [98]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   81,    20],
       [ 2368, 14736]])

In [99]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.80      0.86      0.06      0.83      0.69       101
   low_risk       1.00      0.86      0.80      0.93      0.83      0.70     17104

avg / total       0.99      0.86      0.80      0.92      0.83      0.70     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.

In [ ]:
See README file in the homework Github folder